## Modeling video data with 2DCNN for frame and 1DCNN for time

Need output data from <b>01 processing videos</b>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Attention, Flatten, Reshape, Concatenate, Conv2D, MaxPool2D, Conv1D, MaxPool1D, BatchNormalization
from tensorflow.keras.regularizers import *
from tensorflow import math
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
import tensorflow.keras.backend as kr
import pickle
from data_split import train_valid_test_split

### Load videos

In [ ]:
####
#parameters
test_rate = 0.15
valid_rate = 0.15
window = 30 #frame window
seed = 123456

####
#load focus videos
of = open('processed_videos_160.obj', 'rb')
data, labels = pickle.load(of)
of.close()

####
#transform classes
from sklearn.preprocessing import OrdinalEncoder
label_nums = OrdinalEncoder().fit_transform(labels)

####
#split into train/valid/test
X_train,X_valid,X_test,y_train,y_valid,y_test = train_valid_test_split(data,label_nums,test_rate,valid_rate,window,seed)

<h3>Model building</h3>

In [ ]:
####
#parameters
face_cnn_layers = [[8,4,4], [8,4,4],[8,4,4],[4,4,4]] #2D CNN to model facial data
cnn_layers = [[128,3], [64,3]] #1D CNN to model 2D CNN outputs
out_dense = [64] #output dense hidden size
dropout = False
droprate = 0.4

####
#functional codes
face_in = tf.keras.Input(shape=(window,160,160,1))
x = face_in
#Face 2dCNN blocks
x = kr.reshape(x, (-1,x1.shape[2],x1.shape[3],x1.shape[4]))
for fcnn in face_cnn_layers:
    x = Conv2D(fcnn[0],(fcnn[1],fcnn[2]), activation="relu")(x)
    x = MaxPool2D([2,2], padding='same')(x)
    x = BatchNormalization()(x)
    if dropout:
        x = Dropout(droprate)(x)
x = kr.reshape(x, (-1,window,x.shape[1]*x1.shape[2]*x1.shape[3]))
x = Dense(x.shape[2]//4, activation='relu')(x)
# 1D-CNN Block
for cnn in cnn_layers:    
    x = Conv1D(cnn[0], cnn[1], activation="relu")(x)
    x = MaxPool1D(2, padding='same')(x)
    x = BatchNormalization()(x)
    if dropout:
        x = Dropout(droprate)(x)
# Self-Attention Block
x = Attention()([x, x])
x = tf.keras.layers.GlobalMaxPooling1D()(x)
# Output Block
for d in out_dense:
    x = Dense(d, activation="relu")(x)
    if dropout:
        x = Dropout(droprate)(x)
output = Dense(units=1, activation='softmax')(x)
# Build and Compile model
model = Model(inputs=[face_in], outputs=[output])
model.summary()

### Training

In [ ]:
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=[X_valid, y_valid], batch_size=256, epochs=1000)

### Testing

In [ ]:
model.evaluate(X_test, y_test)